In [1]:
!pip install --upgrade google-cloud-aiplatform google-cloud-logging --quiet
!pip install "google-cloud-aiplatform[evaluation]" --quiet

In [2]:
import pandas as pd
import logging
import google.cloud.logging
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from vertexai.evaluation import (
    MetricPromptTemplateExamples,
    EvalTask,
    PairwiseMetric,
    PointwiseMetric,
)

# Do not remove logging section
client = google.cloud.logging.Client()
client.setup_logging()

pd.set_option("display.max_colwidth", None)

In [3]:
PROJECT_ID = "qwiklabs-gcp-03-3e4e70962c05"
LOCATION = "us-central1"

import vertexai

# Initialize vertexai
vertexai.init()

# Do not remove logging section
log_message = f"Vertex AI initialize: {vertexai}"
logging.info(log_message)

INFO:root:Vertex AI initialize: <module 'vertexai' from '/usr/local/lib/python3.11/dist-packages/vertexai/__init__.py'>


## Exploring example data and making a simple summarization request to a Gen AI model
Setting up some sample product description data for a large language model to summarize and then use a prompt to send a simple summarization request to the model. This configuration will then be used for evaluating the models performance in providing summarizations.

In [4]:
prompt_template="# System_prompt\n{system_prompt} # Question\n{question} # Description {description}"

In [5]:
system_prompt=["You are an retail domestic merchandise expert"]

question=["Provide a one sentence summary of the following text"]

description=[
  "Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated",

  "Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.",

  "Outdoor Garden Furniture Transform your backyard into a personal oasis with this elegant garden furniture set designed for comfort, durability, and timeless style. Whether you're hosting a summer soirée, enjoying a quiet morning coffee, or stretching out under the stars, this set offers the perfect blend of relaxation and sophistication. Crafted with weather-resistant materials and plush, all-season cushions, it’s built to withstand the elements while keeping you cozy. The modern, neutral design complements any garden, patio, or balcony — making it a seamless fit for both small spaces and open-air retreats.",

  "OLED 4K Ultra HD Smart TV. Step into the future of home entertainment with breathtaking clarity, vibrant color, and cinematic sound. This OLED 4K Ultra HD Smart TV transforms your living room into a private theater — where every scene comes alive with lifelike detail and stunning contrast. With self-lit pixels that turn on and off individually, OLED delivers perfect blacks and infinite contrast, revealing depth and dimension that LED TVs simply can’t match. Whether you're streaming your favorite show, gaming at high frame rates, or watching the big game in dazzling clarity, every moment is immersive. Powered by a next-gen AI processor, it automatically adjusts picture and sound based on your environment and content, so you always get the optimal viewing experience. And with an ultra-slim bezel and sleek modern design, it’s not just a TV — it’s a centerpiece.",

  "Smartwash Dishwasher. Let your kitchen work for you. Say goodbye to scrubbing and soaking — the SmartWash Dishwasher delivers a powerful, whisper-quiet clean that saves you time, energy, and water. Whether it’s post-dinner chaos or a pile of party plates, this dishwasher handles it all with precision and polish. With advanced spray technology, high-temperature sanitization, and intuitive smart cycles, every dish comes out sparkling — from delicate wine glasses to stubborn, baked-on pots. The sleek stainless steel finish adds a modern touch to any kitchen, while the adjustable racks and spacious interior let you load more in fewer cycles. Plus, with smart connectivity, you can monitor and control your wash right from your phone — because clean dishes should never slow you down. "
]


In [6]:
# Instantiate a flash_model variable with a GenerativeModel that uses Gemini version gemini-2.0-flash.
flash_model = GenerativeModel(model_name="gemini-2.0-flash")

# Add a generation configuration to the model to set the temperature to 0.
flash_model.generation_config = GenerationConfig(temperature=0.0)

In [7]:
prompt = prompt_template.format(system_prompt=system_prompt,question=question, description=description[1])

In [8]:
llm_response = flash_model.generate_content(prompt)

# Wrap the response text with Markdown() for rendering
markdown_output = Markdown(llm_response.text)

log_message = f"Markdown output: {llm_response.text}"
logging.info(log_message)

markdown_output

INFO:root:Markdown output: This floral midi dress is a lightweight, breathable, and flattering option for summer, featuring a vibrant print, cinched waist, and delicate details.



This floral midi dress is a lightweight, breathable, and flattering option for summer, featuring a vibrant print, cinched waist, and delicate details.


## Prepare the Evaluation Dataset, EvalTask and evaluate of a single model

Setting up the data and scoring method to evaluate the model.

In [9]:
dataset = pd.DataFrame(
    {
        "text": [
            "The Amazon rainforest is the largest rainforest in the world, covering much of northwestern Brazil and extending into Colombia, Peru and other South American countries. It's known for its incredible biodiversity, with millions of species of insects, birds, plants and other organisms. The rainforest plays a crucial role in regulating the Earth's climate by absorbing vast amounts of carbon dioxide.",
            "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower. Constructed from 1887 to 1889 as the entrance to the 1889 World's Fair, it was initially criticized by some of France's leading artists and intellectuals for its design, but it has since become a global cultural icon of France and one of the most recognizable structures in the world."
        ],
        "reference": [
            "The Amazon rainforest is the world's largest rainforest, vital for global climate regulation due to its high biodiversity and carbon absorption.",
            "The Eiffel Tower, a wrought-iron lattice tower in Paris, France, was designed by Gustave Eiffel and built for the 1889 World's Fair, becoming a global cultural icon despite initial criticism."
        ],
        "system_prompt": [
            "You are an expert summarizer. Provide a concise summary of the given text.",
            "You are an expert summarizer. Provide a concise summary of the given text."
        ],
        "question": [
            "Summarize the provided text.",
            "Summarize the provided text."
        ]
    }
)

In [10]:
# Create an EvalTask()
POINTWISE_METRIC = PointwiseMetric(
    metric="custom_text_quality",
    metric_prompt_template=MetricPromptTemplateExamples.Pointwise.SUMMARIZATION_QUALITY,
)

In [11]:
pointwise_eval_task = EvalTask(
    dataset=dataset,
    metrics=[POINTWISE_METRIC],
    experiment="product-summarization-quality",
)

In [12]:
pointwise_result = pointwise_eval_task.evaluate(
    model=flash_model,
    prompt_template=f"# System_prompt\n{system_prompt} # Question\n{question} # Description {description}",
)

INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': '# System_prompt\n[\'You are an retail domestic merchandise expert\'] # Question\n[\'Provide a one sentence summary of the following text\'] # Description ["Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men\'s blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you\'re headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated", \'Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from li

In [13]:
def display_eval_report(eval_result: pd.DataFrame) -> None:
    """Display the evaluation results."""
    metrics_df = pd.DataFrame.from_dict(eval_result.summary_metrics, orient="index").T
    display(Markdown("### Summary Metrics"))
    display(metrics_df)

    display(Markdown("### Row-wise Metrics"))
    display(eval_result.metrics_table)

In [14]:
display_eval_report(pointwise_result)

### Summary Metrics

,row_count,custom_text_quality/mean,custom_text_quality/std
0,2.0,0.0,0.0


### Row-wise Metrics

,text,reference,system_prompt,question,prompt,response,custom_text_quality/explanation,custom_text_quality/score
0,"The Amazon rainforest is the largest rainforest in the world, covering much of northwestern Brazil and extending into Colombia, Peru and other South American countries. It's known for its incredible biodiversity, with millions of species of insects, birds, plants and other organisms. The rainforest plays a crucial role in regulating the Earth's climate by absorbing vast amounts of carbon dioxide.","The Amazon rainforest is the world's largest rainforest, vital for global climate regulation due to its high biodiversity and carbon absorption.",You are an expert summarizer. Provide a concise summary of the given text.,Summarize the provided text.,"# System_prompt\n['You are an retail domestic merchandise expert'] # Question\n['Provide a one sentence summary of the following text'] # Description [""Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated"", 'Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.', ""Outdoor Garden Furniture Transform your backyard into a personal oasis with this elegant garden furniture set designed for comfort, durability, and timeless style. Whether you're hosting a summer soirée, enjoying a quiet morning coffee, or stretching out under the stars, this set offers the perfect blend of relaxation and sophistication. Crafted with weather-resistant materials and plush, all-season cushions, it’s built to withstand the elements while keeping you cozy. The modern, neutral design complements any garden, patio, or balcony — making it a seamless fit for both small spaces and open-air retreats."", ""OLED 4K Ultra HD Smart TV. Step into the future of home entertainment with breathtaking clarity, vibrant color, and cinematic sound. This OLED 4K Ultra HD Smart TV transforms your living room into a private theater — where every scene comes alive with lifelike detail and stunning contrast. With self-lit pixels that turn on and off individually, OLED delivers perfect blacks and infinite contrast, revealing depth and dimension that LED TVs simply can’t match. Whether you're streaming your favorite show, gaming at high frame rates, or watching the big game in dazzling clarity, every moment is immersive. Powered by a next-gen AI processor, it automatically adjusts picture and sound based on your environment and content, so you always get the optimal viewing experience. And with an ultra-slim bezel and sleek modern design, it’s not just a TV — it’s a centerpiece."", 'Smartwash Dishwasher. Let your kitchen work for you. Say goodbye to scrubbing and soaking — the SmartWash Dishwasher delivers a powerful, whisper-quiet clean that saves you time, energy, and water. Whether it’s post-dinner chaos or a pile of party 

## Evaluation and comparison of 2 models

We will compare the summarization capabilities of two different Gemini models: `gemini-2.0-flash` and `gemini-2.0-flash-lite`. This comparison is crucial for informed model selection, allowing us to understand which model performs better for our specific summarization needs.

We will employ **Pairwise Summarization Quality** as our evaluation metric. For this comparison, `gemini-2.0-flash-lite` will serve as our **baseline model**, and `gemini-2.0-flash` will be our **candidate model**. The evaluation will use the same structured Pandas DataFrame that was prepared for the single model evaluation.

The Vertex AI `EvalTask` will orchestrate the generation of responses from both models and then use a human-preference-aligned judge model to compare their summarization quality.

In [15]:
flash_lite_model = GenerativeModel(model_name="gemini-2.0-flash-lite")
flash_lite_model.generation_config = GenerationConfig(temperature=0.0)

In [19]:
PAIRWISE_METRIC = PairwiseMetric(
    metric="pairwise_summarization_quality",
    metric_prompt_template=MetricPromptTemplateExamples.Pairwise.SUMMARIZATION_QUALITY,
    baseline_model=flash_lite_model
    )

pairwise_eval_task = EvalTask(
    dataset=dataset,
    metrics=[PAIRWISE_METRIC],
    experiment="pairwise-product-summarization-quality",
)

pairwise_result = pairwise_eval_task.evaluate(
    prompt_template=f"# System_prompt\n{system_prompt} # Question\n{question} # Description {description}",
    model=flash_model
)

INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': '# System_prompt\n[\'You are an retail domestic merchandise expert\'] # Question\n[\'Provide a one sentence summary of the following text\'] # Description ["Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men\'s blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you\'re headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated", \'Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from li

In [20]:
display_eval_report(pairwise_result)

### Summary Metrics

,row_count,pairwise_summarization_quality/candidate_model_win_rate,pairwise_summarization_quality/baseline_model_win_rate
0,2.0,0.0,0.0


### Row-wise Metrics

,text,reference,system_prompt,question,prompt,response,baseline_model_response,pairwise_summarization_quality/explanation,pairwise_summarization_quality/pairwise_choice
0,"The Amazon rainforest is the largest rainforest in the world, covering much of northwestern Brazil and extending into Colombia, Peru and other South American countries. It's known for its incredible biodiversity, with millions of species of insects, birds, plants and other organisms. The rainforest plays a crucial role in regulating the Earth's climate by absorbing vast amounts of carbon dioxide.","The Amazon rainforest is the world's largest rainforest, vital for global climate regulation due to its high biodiversity and carbon absorption.",You are an expert summarizer. Provide a concise summary of the given text.,Summarize the provided text.,"# System_prompt\n['You are an retail domestic merchandise expert'] # Question\n['Provide a one sentence summary of the following text'] # Description [""Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated"", 'Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.', ""Outdoor Garden Furniture Transform your backyard into a personal oasis with this elegant garden furniture set designed for comfort, durability, and timeless style. Whether you're hosting a summer soirée, enjoying a quiet morning coffee, or stretching out under the stars, this set offers the perfect blend of relaxation and sophistication. Crafted with weather-resistant materials and plush, all-season cushions, it’s built to withstand the elements while keeping you cozy. The modern, neutral design complements any garden, patio, or balcony — making it a seamless fit for both small spaces and open-air retreats."", ""OLED 4K Ultra HD Smart TV. Step into the future of home entertainment with breathtaking clarity, vibrant color, and cinematic sound. This OLED 4K Ultra HD Smart TV transforms your living room into a private theater — where every scene comes alive with lifelike detail and stunning contrast. With self-lit pixels that turn on and off individually, OLED delivers perfect blacks and infinite contrast, revealing depth and dimension that LED TVs simply can’t match. Whether you're streaming your favorite show, gaming at high frame rates, or watching the big game in dazzling clarity, every moment is immersive. Powered by a next-gen AI processor, it automatically adjusts picture and sound based on your environment and content, so you always get the optimal viewing experience. And with an ultra-slim bezel and sleek modern design, it’s not just a TV — it’s a centerpiece."", 'Smartwash Dishwasher. Let your kitchen work for you. Say goodbye to scrubbing and soaking — the SmartWash Dishwasher delivers a powerful, whisper-quiet clean that saves you time, energy, and w

In [23]:
# Display the summary_metrics and metrics_table properties of the result.
print("\n--- Pairwise Summary Metrics ---")
print(pairwise_result.summary_metrics)

print("\n--- Pairwise Metrics Table ---")
print(pairwise_result.metrics_table)


--- Pairwise Summary Metrics ---
{'row_count': 2, 'pairwise_summarization_quality/candidate_model_win_rate': np.float64(0.0), 'pairwise_summarization_quality/baseline_model_win_rate': np.float64(0.0)}

--- Pairwise Metrics Table ---
                                                                                                                                                                                                                                                                                                                                                                                                                                                                              text  \
0                                                                  The Amazon rainforest is the largest rainforest in the world, covering much of northwestern Brazil and extending into Colombia, Peru and other South American countries. It's known for its incredible biodiversity, with millions of